# Summary

## Result

+ Number of customers predicted by the model as “will purchase”: 1,714
+ Number of correct “will purchase” predictions (True Positives): 112

## Metrics Explanation

Meaning in Recommendation Systems
+ Recall (Class 1)
Among all users who actually made a purchase, the proportion correctly identified by the model.
+ Precision (Class 1)
Among all users recommended by the model, the proportion who actually made a purchase.
+ F1-score
A combined measure of recommendation capability that balances both precision and recall.
+ AUC (optional)
Overall performance across different classification thresholds (more robust for imbalanced datasets).

## Analysis of Actual Promotion Cost vs. Effectiveness

Suppose a company have 100,000 existing users and select 1,714 of them for targeted recommendations or email promotions:

🌟 Effectiveness Evaluation:

For every 100 users recommended, about 6–7 will make a purchase (precision).

This is a fairly common baseline:

+ For most e-commerce cold-start recommendations, the click-through rate (CTR) is below 1%, and the conversion rate (CVR) is below 5%.
+ Your model’s recommended conversion rate is around 6–7%, significantly higher than random.

## Highlights

1. Two-Tower Retrieval Recommendation Model: Built a recall system incorporating user/item embeddings, enriched with contextual information such as time, location, and company.
2.	New Product Cold-Start Prediction Task: Modeled purchase likelihood for unseen products by reusing user embeddings in combination with an MLP and Focal Loss.
3.	Data Engineering & Sample Construction: Generated positive/negative samples from historical user behaviors, addressed class imbalance, and evaluated model performance using metrics such as HitRate@K, Precision, and Recall.
4.	Comparative Analysis with Multiple Models: Evaluated and compared models including Logistic Regression, MLP, and PyTorch-based implementations, clearly demonstrating the cold-start challenges and the impact of model choice on results.
5.	High Extensibility: The project can be further extended to support ranking models, item-to-item recall, and advanced features such as multimodal inputs (product text/images).

# 1. Get data
read data and clean it.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir("../data/")

['May-19',
 'indoor_garden_preferencere_sponses.csv',
 '.DS_Store',
 'List Export 2025-04-08.csv',
 'df_until_May_19.csv',
 'May-15',
 'category_df_fillna.csv',
 'df_user_profile.csv',
 'Bundle-Orders-2025-5-16.csv',
 'orders_export_all.csv',
 'orders_export_2.csv',
 'orders_export_1.csv',
 'landingpage-2025-04-01-2025-05-23.csv']

In [3]:
df = pd.read_csv("../data/df_until_May_19.csv")

In [4]:
df["Created at"].max()

'2025-05-19 09:28:55-05:00'

In [5]:
df.shape

(705801, 31)

In [6]:
df["orderid"].nunique()

389050

In [7]:
df.columns

Index(['orderid', 'Email', 'Lineitem sku', 'Lineitem name', 'Lineitem price',
       'Subtotal', 'Shipping', 'Taxes', 'Total', 'Discount Amount',
       'Created at', 'Lineitem quantity', 'Lineitem compare at price',
       'Shipping Name', 'Shipping Street', 'Shipping Address1',
       'Shipping Address2', 'Shipping Company', 'Shipping City',
       'Shipping Zip', 'Shipping Province', 'Shipping Country',
       'Shipping Phone', 'Vendor', 'Year', 'Month', 'Day', 'Hour', 'type name',
       'Parent_SKU', 'Product Type'],
      dtype='object')

In [8]:
# df["Product Type"]

In [9]:
df["Created at"] = pd.to_datetime(df["Created at"], utc=True)
# 将 UTC 时间转换为美国中部时间（CST/CDT）
df["Created at"] = df["Created at"].dt.tz_convert('America/Chicago')

In [10]:
# df["Shipping Country"].unique()

In [11]:
df = df[df["Shipping Country"]=="US"]

In [12]:
df.shape

(700915, 31)

# 2. Plan Description

## 1.Two-Tower Retrieval Model: (Completed)

+ Left Tower (User Tower): Email (can be encoded as user_id), shipping country/province/city, time features (year, month, day, hour), etc.
+ Right Tower (Item Tower): Lineitem SKU, lineitem name, vendor, product type, price, discount, etc.

Used to construct a user–item embedding space and perform retrieval by computing vector similarities.

## 2.Ranking Model: (Not applied due to limited number of products, use simpler model instead)

Built on top of the retrieval stage, a ranking model would leverage richer features such as:
+ User-side: Address, order time, historical behavioral statistics (e.g., number of orders in the past 30 days)
+ Item-side: Price, discount, category, sales volume, etc.

Possible model architectures include Wide & Deep, DNN, DCN, and DIN, and TFRS also supports custom ranking models.

## 3. Sequential Modeling (Optional)
+ Construct click/purchase sequences for each user, using models such as:
+ GRU4Rec, SASRec, or Transformer-based recommendation models
+ TFRS also supports this type of RNN-based model.

# 3. Preparation for verification data 
+ find the new product launch time
+ thus can use the model and data previous this launch time to predict which old user will buy the new product
+ then verify it with the real data after the lauch time

## used: check new product lanuch time

In [21]:
import pandas as pd

# 确保 'Created at' 是 datetime 格式
df['Created at'] = pd.to_datetime(df['Created at'])

# 分组聚合最早和最晚时间
product_lifecycle = df.groupby('type name')['Created at'].agg(
    first_order_date='min',
    last_order_date='max'
).reset_index()

# 可选：计算生命周期长度（单位：天）
product_lifecycle['days_between'] = (product_lifecycle['last_order_date'] - product_lifecycle['first_order_date']).dt.days

In [63]:
# product_lifecycle

In [64]:
# product_lifecycle["days_between"].hist()

In [33]:
df[df["type name"]=="EZ Self-Watering Herb Planter Box with Trellis"]["Lineitem quantity"].sum()

3346

In [35]:
# 3346/120

In [36]:
df[df["type name"]=="EZ Self-Watering Herb Planter Box with Trellis"]["Total"].sum()

1703727.01

In [30]:
# df[df["type name"]=="EZ Self-Watering Mini Planter Pot with Trellis"]["Total"].sum()

1737708.67

In [32]:
# df[df["type name"]=="EZ Self-Watering Tomato Planter with Trellis"]["Total"].sum()

In [37]:
new_products=["EZ Self-Watering Herb Planter Box with Trellis"
"EZ Self-Watering Mini Planter Pot with Trellis"
"EZ Self-Watering Tomato Planter with Trellis"]

In [38]:
product_lifecycle[product_lifecycle["first_order_date"]>"2025-01-01 09:09:04-05:00"].sort_values(by="days_between",ascending=False).head(10)

,type name,first_order_date,last_order_date,days_between
493,EZ Self-Watering Herb Planter Box with Trellis,2025-01-14 07:08:36-06:00,2025-05-19 08:44:03-05:00,125
496,EZ Self-Watering Mini Planter Pot with Trellis,2025-01-14 13:41:23-06:00,2025-05-19 09:00:01-05:00,124
497,EZ Self-Watering Tomato Planter with Trellis,2025-01-15 08:19:27-06:00,2025-05-19 08:36:43-05:00,123
1417,Vego Irrigation Kit,2025-01-20 21:13:37-06:00,2025-05-19 09:28:52-05:00,118
495,EZ Self-Watering Home Planter,2025-01-25 15:31:07-06:00,2025-05-18 19:18:11-05:00,113
1184,Shipping Protection by Route,2025-01-29 02:57:32-06:00,2025-05-19 09:28:55-05:00,110
519,Elevated garden bed wheel set,2025-02-01 05:04:40-06:00,2025-05-18 14:23:27-05:00,106
715,Improved Meyer Lemon Bush,2025-02-02 17:30:49-06:00,2025-05-18 13:07:32-05:00,104
1010,Plant 'n' Pop Seed Bundle,2025-02-05 12:35:38-06:00,2025-05-19 09:08:02-05:00,102
1162,Seed Starting Bundle with Lid,2025-02-07 04:43:06-06:00,2025-05-18 22:27:33-05:00,100


In [39]:
# product_lifecycle.sort_values(by="days_between",ascending=False).head(60)

In [40]:
# product_lifecycle.sort_values(by=["first_order_date",ascending=False).head(60)

## new product

In [13]:
df[df["type name"]=="EZ Self-Watering Herb Planter Box with Trellis"]["Lineitem quantity"].sum()

3341

In [14]:
df[df["type name"]=="EZ Self-Watering Herb Planter Box with Trellis"]["Total"].sum()

1701982.02

## split data, before and after launch time

In [15]:
df_before = df[df["Created at"]<"2025-01-14 01:08:36-06:00"]

In [16]:
df_before["Created at"].min(), df_before["Created at"].max()

(Timestamp('2020-09-29 18:47:42-0500', tz='America/Chicago'),
 Timestamp('2025-01-14 00:49:49-0600', tz='America/Chicago'))

In [17]:
df_after = df[df["Created at"]>"2025-01-15 01:08:36-06:00"]

In [18]:
df_after["Created at"].min(), df_after["Created at"].max()

(Timestamp('2025-01-15 01:51:45-0600', tz='America/Chicago'),
 Timestamp('2025-05-19 09:28:55-0500', tz='America/Chicago'))

# 4. Model and Data structure Definition

In [19]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text
import numpy as np

In [20]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

print(tf.__version__)
print(tfrs.__version__)

2.14.0
v0.7.3


## dataset

In [62]:
# 1. 保证字段是 string 类型
df_before["hour"] = df_before["Hour"].astype(str)
df_before["month"] = df_before["Month"].astype(str)

# 2. 拟合价格归一化器
price_norm = tf.keras.layers.Normalization(axis=None)
price_norm.adapt(df_before["Lineitem price"].values.astype("float32"))

# 3. 创建原始特征的 Dataset
features_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": df_before["Email"].astype(str),
    "province": df_before["Shipping Province"].astype(str),
    "city": df_before["Shipping City"].astype(str),
    "company": df_before["Shipping Company"].astype(str),
    "hour": df_before["hour"],
    "month": df_before["month"],
    "item_id": df_before["Lineitem name"].astype(str),
    "type": df_before["type name"].astype(str),
    "price": df_before["Lineitem price"].astype("float32"),
})

# 4. 转换为 TFRS 模型所需格式：(features_dict, item_id)
# 注意这里的标签为 item_id，用于 Retrieval 学习目标
train_ds = features_ds.map(lambda x: (x, x["item_id"]))

# 5. Shuffle, batch, prefetch
train_ds = train_ds.shuffle(100_000).batch(1024).prefetch(tf.data.AUTOTUNE)

/var/folders/h1/kkr0d8_n4414qhwr_pl231w00000gn/T/ipykernel_53326/2190089378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_before["hour"] = df_before["Hour"].astype(str)
/var/folders/h1/kkr0d8_n4414qhwr_pl231w00000gn/T/ipykernel_53326/2190089378.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_before["month"] = df_before["Month"].astype(str)


In [42]:
# candidate_ds = item_ds.apply(tf.data.experimental.unique())

In [34]:
train_ds

<_PrefetchDataset element_spec={'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'province': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'city': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'company': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'hour': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'month': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'item_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'price': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>

## two tower model

In [78]:
class TwoTowerModel(tfrs.models.Model):
    def __init__(self, user_model, item_model, candidate_ds):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=candidate_ds.batch(128).map(item_model))
        )

    def compute_loss(self, features, training=False):
        user_features, item_id = features  # 解包 tuple
    
        user_embedding = self.user_model({
            "user_id": user_features["user_id"],
            "province": user_features["province"],
            "city": user_features["city"],
            "company": user_features["company"],
            "hour": user_features["hour"],
            "month": user_features["month"],
        })
    
        item_embedding = self.item_model({
            "item_id": item_id,
            "type": user_features["type"],
            "price": user_features["price"],
        })
    
        return self.task(user_embedding, item_embedding)

    # 添加 call 方法，让 Keras 知道前向结构
    def call(self, features):
        return {
            "user_embedding": self.user_model({
                "user_id": features["user_id"],
                "province": features["province"],
                "city": features["city"],
                "company": features["company"],
                "hour": features["hour"],
                "month": features["month"],
            }),
            "item_embedding": self.item_model({
                "item_id": features["item_id"],
                "type": features["type"],
                "price": features["price"],
            }),
        }

## user tower

In [114]:
class UserModel(tf.keras.Model):
    def __init__(self, user_ids, provinces, cities, companies, hours, months, embedding_dim=32, output_dim=64):
        super().__init__()
        self.user_id_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(user_ids) + 1, embedding_dim)
        ])
        self.province_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=provinces, mask_token=None),
            tf.keras.layers.Embedding(len(provinces) + 1, embedding_dim)
        ])
        self.city_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=cities, mask_token=None),
            tf.keras.layers.Embedding(len(cities) + 1, embedding_dim)
        ])
        self.company_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=companies, mask_token=None),
            tf.keras.layers.Embedding(len(companies) + 1, embedding_dim)
        ])
        self.hour_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=hours, mask_token=None),
            tf.keras.layers.Embedding(len(hours) + 1, embedding_dim)
        ])
        self.month_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=months, mask_token=None),
            tf.keras.layers.Embedding(len(months) + 1, embedding_dim)
        ])

        self.output_layer = tf.keras.layers.Dense(output_dim)  # 投影为统一维度


    def call(self, inputs, training=False, cold_start=False):
        user_id_emb = self.user_id_lookup(inputs["user_id"])
        province_emb = self.province_lookup(inputs["province"])
        city_emb = self.city_lookup(inputs["city"])
        company_emb = self.company_lookup(inputs["company"])
        hour_emb = self.hour_lookup(inputs["hour"])
        month_emb = self.month_lookup(inputs["month"])

        # 模拟 cold start：训练时 Dropout，预测时 cold_start 参数控制
        if training:
            user_id_emb = tf.keras.layers.Dropout(0.3)(user_id_emb)
        elif cold_start:
            user_id_emb = tf.zeros_like(user_id_emb)

        x = tf.concat([
            user_id_emb,
            province_emb,
            city_emb,
            company_emb,
            hour_emb,
            month_emb
        ], axis=1)

        return self.output_layer(x)

## product tower

In [101]:
class ItemModel(tf.keras.Model):
    def __init__(self, item_ids, item_types, embedding_dim=32, output_dim=64):
        super().__init__()
        self.item_id_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=item_ids, mask_token=None),
            tf.keras.layers.Embedding(len(item_ids) + 1, embedding_dim)
        ])
        self.type_lookup = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=item_types, mask_token=None),
            tf.keras.layers.Embedding(len(item_types) + 1, embedding_dim)
        ])
        self.price_norm = tf.keras.layers.Normalization(axis=None)

        self.output_layer = tf.keras.layers.Dense(output_dim)  # 投影为统一维度

    def call(self, inputs):
        x = tf.concat([
            self.item_id_lookup(inputs["item_id"]),
            self.type_lookup(inputs["type"]),
            tf.expand_dims(self.price_norm(inputs["price"]), axis=1),
        ], axis=1)
        return self.output_layer(x)

# 5.Train

In [102]:
seen_user_ids = set(df_before["Email"].astype(str))

In [103]:
unique_user_ids = df["Email"].astype(str).unique().tolist()
provinces = df["Shipping Province"].astype(str).unique().tolist()
cities = df["Shipping City"].astype(str).unique().tolist()
companies = df["Shipping Company"].astype(str).unique().tolist()
hours = df["Hour"].astype(str).unique().tolist()
months = df["Month"].astype(str).unique().tolist()

item_ids = df["Lineitem name"].astype(str).unique().tolist()
item_types = df["type name"].astype(str).unique().tolist()
# item_price = df["Lineitem price"].astype(str).unique().tolist()

In [115]:
user_model = UserModel(unique_user_ids, provinces, cities, companies, hours, months)

In [105]:
item_model = ItemModel(item_ids, item_types)
item_model.price_norm.adapt(df["Lineitem price"].astype("float32").values)

In [106]:
# 用 pandas 先去重，选取 item 特征
item_df = df_before[["Lineitem name", "type name", "Lineitem price"]].drop_duplicates()

# 构建 candidate_ds，包含 item_id, type, price
candidate_ds = tf.data.Dataset.from_tensor_slices({
    "item_id": item_df["Lineitem name"].astype(str).values,
    "type": item_df["type name"].astype(str).values,
    "price": item_df["Lineitem price"].astype("float32").values,
})

In [116]:
model = TwoTowerModel(user_model, item_model, candidate_ds=candidate_ds)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [118]:
# 创建 BruteForce 检索层
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    candidate_ds.batch(128).map(lambda x: (x["item_id"], model.item_model(x)))
)

# 测试一个用户
test_user = {
    "user_id": tf.constant(["some_user_id"]),
    "province": tf.constant(["CA"]),
    "city": tf.constant(["San Francisco"]),
    "company": tf.constant(["Acme Inc"]),
    "hour": tf.constant(["14"]),
    "month": tf.constant(["6"])
}
scores, items = index(test_user)
print("Top recommendations:", items[0, :5].numpy())

Top recommendations: [b"Vego Garden Pacific Greenhouse - 8.5' x 16.5'"
 b"Vego Garden Pacific Greenhouse - 8.5' x 16.5'"
 b"Vego Garden Pacific Greenhouse - 8.5' x 16.5'"
 b"Vego Garden Pacific Greenhouse - 8.5' x 12.5'"
 b"Vego Garden Pacific Greenhouse - 8.5' x 10.5'"]


Stopped the training process manually when no further model improvement was observed.

In [120]:
model.fit(train_ds, epochs=3)

Epoch 1/3
472/472 [==============================] - 127s 268ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0816 - factorized_top_k/top_5_categorical_accuracy: 0.2018 - factorized_top_k/top_10_categorical_accuracy: 0.2913 - factorized_top_k/top_50_categorical_accuracy: 0.6513 - factorized_top_k/top_100_categorical_accuracy: 0.7783 - loss: 3668.3153 - regularization_loss: 0.0000e+00 - total_loss: 3668.3153
Epoch 2/3
472/472 [==============================] - 128s 271ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0873 - factorized_top_k/top_5_categorical_accuracy: 0.2234 - factorized_top_k/top_10_categorical_accuracy: 0.3209 - factorized_top_k/top_50_categorical_accuracy: 0.6984 - factorized_top_k/top_100_categorical_accuracy: 0.8198 - loss: 3475.8531 - regularization_loss: 0.0000e+00 - total_loss: 3475.8531
Epoch 3/3
 42/472 [=>............................] - ETA: 1:55 - factorized_top_k/top_1_categorical_accuracy: 0.0702 - factorized_top_k/top_5_categorical_accuracy

KeyboardInterrupt: 

In [71]:
model.evaluate(train_ds)

472/472 [==============================] - 114s 240ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0677 - factorized_top_k/top_5_categorical_accuracy: 0.2839 - factorized_top_k/top_10_categorical_accuracy: 0.4216 - factorized_top_k/top_50_categorical_accuracy: 0.8063 - factorized_top_k/top_100_categorical_accuracy: 0.8935 - loss: 3035.4466 - regularization_loss: 0.0000e+00 - total_loss: 3035.4466


[0.06774871796369553,
 0.28385159373283386,
 0.4216342866420746,
 0.8062613010406494,
 0.8935382962226868,
 1991.751953125,
 0,
 1991.751953125]

# 6. Model save and load

In [80]:
# # 保存权重（包括 user_model 和 item_model 的所有参数）
# model.save_weights("retrieval_weights")

# # 单独保存 user_model
# model.user_model.save("user_model")
# model.item_model.save("item_model")

In [75]:
# # 加载 user/item 塔
# loaded_user_model = tf.keras.models.load_model("user_model")
# loaded_item_model = tf.keras.models.load_model("item_model")

# # 重新构建 TwoTowerModel 并加载权重
# model = TwoTowerModel(loaded_user_model, loaded_item_model, candidate_ds)
# model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
# model.load_weights("retrieval_weights")

# 7. Analysis model performance

## Check the new user prediction ratio

In [82]:
new_user_ids = set(df_after["Email"]) - set(df_before["Email"])
df_new_users = df_after[df_after["Email"].isin(new_user_ids)]

In [90]:
df_after["Email"].nunique(),df_before["Email"].nunique(),

(71454, 191264)

In [92]:
len(new_user_ids)

54860

In [86]:
df_new_users.shape

(166472, 31)

In [95]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    candidate_ds.batch(256).map(lambda x: (x["item_id"], model.item_model(x)))
)

In [121]:
import random
from tqdm import tqdm

top_k = 10

# 提取候选商品的 embedding 和 id
all_item_embs = []
all_item_ids = []

for batch in candidate_ds.batch(128):
    all_item_embs.append(model.item_model(batch))
    all_item_ids.append(batch["item_id"])

item_embeddings = tf.concat(all_item_embs, axis=0)
item_ids = tf.concat(all_item_ids, axis=0)

# 随机采样新用户
sample_emails = random.sample(
    df_new_users["Email"].unique().tolist(),
    len(df_new_users["Email"].unique()) // 30
)

hit_count = 0
total_count = 0

for user_id in tqdm(sample_emails):
    user_df = df_new_users[df_new_users["Email"] == user_id]
    if user_df.empty:
        continue

    user_dict = {
        "user_id": tf.constant([user_id]),
        "province": tf.constant([str(user_df["Shipping Province"].iloc[0])]),
        "city": tf.constant([str(user_df["Shipping City"].iloc[0])]),
        "company": tf.constant([str(user_df["Shipping Company"].iloc[0])]),
        "hour": tf.constant([str(user_df["Hour"].iloc[0])]),
        "month": tf.constant([str(user_df["Month"].iloc[0])]),
    }

    try:
        # 手动获取冷启动的 user embedding（关闭 user_id）
        user_emb = model.user_model(user_dict, cold_start=True)  # ✅ 使用 cold_start=True
        scores = tf.linalg.matmul(user_emb, item_embeddings, transpose_b=True)
        top_scores, top_indices = tf.math.top_k(scores, k=top_k)

        topk_items = set([x.numpy().decode("utf-8") for x in tf.gather(item_ids, top_indices[0])])
        true_items = set(user_df["Lineitem name"].astype(str))

        if topk_items & true_items:
            hit_count += 1
        total_count += 1
    except Exception as e:
        print(f"Error for user {user_id}: {e}")
        continue

hit_rate = hit_count / total_count if total_count > 0 else 0
print(f"ColdStart HitRate@{top_k}: {hit_rate:.4f}  ({hit_count}/{total_count})")

100%|███████████████████████████████████████| 1828/1828 [03:09<00:00,  9.66it/s]

ColdStart HitRate@10: 0.0055  (10/1828)


+ HitRate@10: 0.0159  (87/5486)


ColdStart HitRate@10: 0.0055  (10/1828)

## Old user purchased prediction

In [122]:
import random
from tqdm import tqdm

top_k = 10
hit_count = 0
total_count = 0

# 找出出现在两个数据集中的用户
common_emails = list(set(df_after["Email"]) & set(df_before["Email"]))

In [123]:
len(common_emails)

16594

In [126]:
# 你可以只采样一部分用于测试
sample_emails = random.sample(common_emails, len(common_emails) // 10)

In [127]:
len(sample_emails)

1659

In [128]:
for user_id in tqdm(sample_emails):
    # 取旧数据中该用户的画像（假设它作为模型输入）
    user_df_before = df_before[df_before["Email"] == user_id]
    user_df_after = df_after[df_after["Email"] == user_id]

    if user_df_before.empty or user_df_after.empty:
        continue

    # 构造用户画像（使用 before 中的信息）
    user_dict = {
        "user_id": tf.constant([user_id]),
        "province": tf.constant([str(user_df_before["Shipping Province"].iloc[0])]),
        "city": tf.constant([str(user_df_before["Shipping City"].iloc[0])]),
        "company": tf.constant([str(user_df_before["Shipping Company"].iloc[0])]),
        "hour": tf.constant([str(user_df_before["Hour"].iloc[0])]),
        "month": tf.constant([str(user_df_before["Month"].iloc[0])]),
    }

    try:
        # 获取推荐商品 Top-K
        scores, items = index(user_dict)
        topk_items = set([x.decode("utf-8") for x in items[0, :top_k].numpy()])

        # 真实的新购买商品
        true_items = set(user_df_after["Lineitem name"].astype(str))

        if topk_items & true_items:
            hit_count += 1
        total_count += 1

    except Exception as e:
        print(f"Error for user {user_id}: {e}")
        continue

hit_rate = hit_count / total_count if total_count > 0 else 0
print(f"Transfer HitRate@{top_k}: {hit_rate:.4f}  ({hit_count}/{total_count})")

100%|███████████████████████████████████████| 1659/1659 [03:07<00:00,  8.84it/s]

Transfer HitRate@10: 0.0199  (33/1659)


## For a new product: EZ Planter

In [131]:
# 计算 after 和 before 的共同用户
all_old_users = set(df_before["Email"].unique()) & set(df_after["Email"].unique())

In [136]:
# len(df_before["Email"].unique()),len(df_after["Email"].unique())

In [133]:
len(all_old_users)

16594

In [137]:
# 获取共同用户
common_users = list(set(df_before["Email"]) & set(df_after["Email"]))

# 提取每个字段的值（取第一行）
user_df_before = df_before[df_before["Email"].isin(common_users)]
user_features = user_df_before.groupby("Email").first().reset_index()

# 构造 batch 输入
user_dict = {
    "user_id": tf.constant(user_features["Email"].astype(str).tolist()),
    "province": tf.constant(user_features["Shipping Province"].astype(str).tolist()),
    "city": tf.constant(user_features["Shipping City"].astype(str).tolist()),
    "company": tf.constant(user_features["Shipping Company"].astype(str).tolist()),
    "hour": tf.constant(user_features["Hour"].astype(str).tolist()),
    "month": tf.constant(user_features["Month"].astype(str).tolist()),
}

# 一次性获取 embedding
user_embeddings = model.user_model(user_dict).numpy()

# 映射回 user_id -> embedding
user_vectors = dict(zip(user_features["Email"], user_embeddings))

In [144]:
df_after.columns

Index(['orderid', 'Email', 'Lineitem sku', 'Lineitem name', 'Lineitem price',
       'Subtotal', 'Shipping', 'Taxes', 'Total', 'Discount Amount',
       'Created at', 'Lineitem quantity', 'Lineitem compare at price',
       'Shipping Name', 'Shipping Street', 'Shipping Address1',
       'Shipping Address2', 'Shipping Company', 'Shipping City',
       'Shipping Zip', 'Shipping Province', 'Shipping Country',
       'Shipping Phone', 'Vendor', 'Year', 'Month', 'Day', 'Hour', 'type name',
       'Parent_SKU', 'Product Type'],
      dtype='object')

In [145]:
import random
from sklearn.model_selection import train_test_split

# 新品名称
new_item = "EZ Self-Watering Herb Planter Box with Trellis"

# 计算 after 和 before 的共同用户
all_old_users = set(df_before["Email"].unique()) & set(df_after["Email"].unique())

# 正样本：老用户中买了新品的
positive_users = df_after[df_after["type name"] == new_item]["Email"].unique()
positive_users = list(set(positive_users) & all_old_users)

# 负样本：老用户中没买过新品的
negative_users = list(all_old_users - set(positive_users))

# 创建有标签的用户列表
all_labeled_users = [(u, 1) for u in positive_users] + [(u, 0) for u in negative_users]

# 按 Email 55% / 45% 划分训练与测试
emails = [u for u, _ in all_labeled_users]
labels = [l for _, l in all_labeled_users]
train_emails, test_emails, train_labels, test_labels = train_test_split(
    emails, labels, test_size=0.45, stratify=labels, random_state=42
)

# 构造训练集和测试集
train_user_labeled = list(zip(train_emails, train_labels))
test_user_labeled = list(zip(test_emails, test_labels))

print(f"Train users: {len(train_user_labeled)}, Test users: {len(test_user_labeled)}")

Train users: 9126, Test users: 7468


In [146]:
import numpy as np

# 过滤掉 user_vectors 中不存在的用户（极个别可能出错或缺省）
train_user_labeled = [(u, l) for u, l in train_user_labeled if u in user_vectors]
test_user_labeled = [(u, l) for u, l in test_user_labeled if u in user_vectors]

# 构建 numpy array 格式数据
X_train = np.array([user_vectors[u] for u, _ in train_user_labeled])
y_train = np.array([l for _, l in train_user_labeled])

X_test = np.array([user_vectors[u] for u, _ in test_user_labeled])
y_test = np.array([l for _, l in test_user_labeled])

In [147]:
from collections import Counter
print("Train label distribution:", Counter(train_labels))
print("Test label distribution:", Counter(test_labels))

Train label distribution: Counter({0: 8689, 1: 437})
Test label distribution: Counter({0: 7110, 1: 358})


### use LogisticRegression

In [149]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# 自动平衡类别权重
clf = LogisticRegression(max_iter=1000, class_weight="balanced")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.584092126405999
              precision    recall  f1-score   support

           0       0.96      0.59      0.73      7110
           1       0.06      0.48      0.10       358

    accuracy                           0.58      7468
   macro avg       0.51      0.53      0.41      7468
weighted avg       0.91      0.58      0.70      7468



+ 当前模型的 recall = 0.48，说明命中了近一半的潜在客户，但 precision = 0.06，说明推荐给太多不买的人了。
+ 比起 accuracy，更应该优化 precision、recall、f1 或 AUC。
+ 比随机推荐强很多才有实用价值：当前 precision = 6%，baseline 是 4.8%，差距很小，需要优化。

+ The current model’s recall is 0.48, indicating that it captures nearly half of the potential customers, but the precision is only 0.06, meaning that it recommends to too many people who are unlikely to purchase.
+ Compared to accuracy, it is more important to optimize precision, recall, F1 score, or AUC.
+ The model must significantly outperform random recommendation to be practically valuable: the current precision is 6%, while the baseline is 4.8%, showing only a small improvement and requiring further optimization.

In [152]:
# from xgboost import XGBClassifier

# clf = XGBClassifier(
#     scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),  # 类别权重
#     use_label_encoder=False,
#     eval_metric="logloss"
# )
# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# y_prob = clf.predict_proba(X_test)[:, 1]

# print(classification_report(y_test, y_pred))

### use mlp

In [155]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return focal_loss.mean()

In [158]:
import numpy as np

# 构造训练集
X_train = np.array([user_vectors[email] for email, _ in train_user_labeled])
y_train = np.array([label for _, label in train_user_labeled])

# 构造测试集
X_test = np.array([user_vectors[email] for email, _ in test_user_labeled])
y_test = np.array([label for _, label in test_user_labeled])

In [169]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 转换为 PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 构建 TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# 构建 DataLoader（batch_size 可调）
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256)

In [166]:
X_train.shape

(9126, 64)

In [163]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets.float(), reduction="none")
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss

In [164]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32]):
        super(MLPClassifier, self).__init__()
        layers = []
        last_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(last_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            last_dim = h
        layers.append(nn.Linear(last_dim, 1))  # 输出是logits
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x).squeeze(1)

In [182]:
from sklearn.metrics import classification_report

def train_and_evaluate(model, train_loader, test_loader, epochs=10, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    pos_weight = torch.tensor([20.0])  # 正类比负类少 20 倍时
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
    # criterion = FocalLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f"Epoch {epoch + 1}: Train Loss = {train_loss / len(train_loader):.4f}")

    # 验证阶段
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            preds = model(xb)
            probs = torch.sigmoid(preds)
            all_preds.extend((probs > 0.5).cpu().numpy())
            all_labels.extend(yb.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, digits=4))
    return all_preds, all_labels

In [208]:
input_dim = X_train.shape[1]
model = MLPClassifier(input_dim=input_dim)

In [209]:
all_preds, all_labels = train_and_evaluate(model, train_loader, test_loader, epochs=10, lr=1e-3)

Epoch 1: Train Loss = 1.3209
Epoch 2: Train Loss = 1.3192
Epoch 3: Train Loss = 1.3017
Epoch 4: Train Loss = 1.2957
Epoch 5: Train Loss = 1.2789
Epoch 6: Train Loss = 1.2671
Epoch 7: Train Loss = 1.2571
Epoch 8: Train Loss = 1.2472
Epoch 9: Train Loss = 1.2176
Epoch 10: Train Loss = 1.2112

Classification Report:
              precision    recall  f1-score   support

         0.0     0.9583    0.5848    0.7264      7110
         1.0     0.0566    0.4944    0.1015       358

    accuracy                         0.5805      7468
   macro avg     0.5074    0.5396    0.4139      7468
weighted avg     0.9151    0.5805    0.6964      7468



In [206]:
# all_preds, all_labels = train_and_evaluate(model, train_loader, test_loader, epochs=10, lr=1e-3)

In [210]:
from sklearn.metrics import confusion_matrix

# 假设 all_labels 和 all_preds 是列表或 numpy 数组
cm = confusion_matrix(all_labels, all_preds)

tn, fp, fn, tp = cm.ravel()

print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")
print(f"模型预测为“会买”的人数: {tp + fp}")
print(f"模型预测对的“会买”的人数 (真正例): {tp}")

True Negatives (TN): 4158
False Positives (FP): 2952
False Negatives (FN): 181
True Positives (TP): 177
模型预测为“会买”的人数: 3129
模型预测对的“会买”的人数 (真正例): 177


In [211]:
len(all_labels)

7468